In [1]:
import cv2
import time
import threading
import numpy as np
import keyboard

In [2]:
from tensorflow.keras.models import load_model

In [3]:
new_model = load_model('./model/basic_model-best-model-2020-10-11_16-41-12.h5')

In [4]:
# !pip list

In [5]:
def KBC(ac):
    pred = int(ac)
    KAC = ['d','a','w','s','[',']']
    # KAC = ['left','right','down','up',']','[']
    keyboard.send(KAC[pred],do_release=False)
    time.sleep(0.25)
    keyboard.release(KAC[pred])
    # if pred == 0:
    #     # Seek backward 10 seconds.
    #     keyboard.send('j')
    #     time.sleep(0.25)
    # elif pred == 1:
    #     # Seek forward 10 seconds.
    #     keyboard.send('l')
    #     time.sleep(0.25)
    # elif pred == 2:
    #     # decrease volume by 5%.
    #     keyboard.send('down')
    #     time.sleep(0.25)
    # elif pred == 3:
    #     # Increase volume by 5%.
    #     keyboard.send('up')
    #     time.sleep(0.25)
    # elif pred == 4:
    #     # Pan right.
    #     keyboard.send('d',do_release=False)
    #     time.sleep(0.25)
    #     keyboard.release('d')
    # elif pred == 5:
    #     # Pan left.
    #     keyboard.send('a',do_release=False)
    #     time.sleep(0.25)
    #     keyboard.release('a')


In [20]:
class ipcamCapture:
    def __init__(self, URL):
        self.Frame = []
        self.Framecube = []	
        self.img = []
        self.status = False
        self.isstop = False
        self.x = 0
        self.flag = False
        
        #connect to webcam
        self.capture = cv2.VideoCapture(URL)

    def start(self):
        #start thread
        print('ipcam started!')
        threading.Thread(target=self.queryframe, daemon=True, args=()).start()

    def stop(self):
        #STOP Flag
        self.isstop = True
        print('ipcam stopped!')
   
    def getframe(self):
        #Get new frame

        return self.Frame
        
    def queryframe(self):
        height, width = 128, 171
        scale = 136
        while (not self.isstop):
            self.status, self.Frame = self.capture.read()
            self.img = self.Frame
            if len(self.Framecube) < 30:
                if self.x % 2==0:

                    self.img = cv2.resize(self.img, (width, height), interpolation = cv2.INTER_LINEAR)
                    off_h = abs(np.ceil((height - scale) / 2).astype(int))
                    off_w = abs(np.ceil((width - scale) / 2).astype(int))
                    self.img = self.img[off_h:off_h + scale, off_w:off_w + scale, :]
                    self.img = cv2.resize(self.img, (112, 112), interpolation = cv2.INTER_LINEAR)

                    self.Framecube.append(self.img)
                    # self.Framecube.append(self.Frame)
                    if self.x >= 1800:
                        self.x = 0
                        self.x += 1
                    else:
                        self.x+=1     
                else:
                    self.x+=1
                
            
            elif len(self.Framecube) >= 30 and len(self.Framecube) < 180:
                if self.x % 2==0:

                    self.img = cv2.resize(self.img, (width, height), interpolation = cv2.INTER_LINEAR)
                    off_h = abs(np.ceil((height - scale) / 2).astype(int))
                    off_w = abs(np.ceil((width - scale) / 2).astype(int))
                    self.img = self.img[off_h:off_h + scale, off_w:off_w + scale, :]
                    self.img = cv2.resize(self.img, (112, 112), interpolation = cv2.INTER_LINEAR)
                    
                    self.Framecube.append(self.img)
                    # self.Framecube.append(self.Frame)
                    if self.x >= 1800:
                        self.x = 0
                        self.x += 1
                    else:
                        self.x+=1     
                else:
                    self.x+=1
                
                if self.flag:
                    del self.Framecube[0:13]
                    # del self.Framecube[0:10]
                    self.flag = False
                else:
                    pass
                
        
        self.capture.release()
    def getcubeframe(self):
        if len(self.Framecube) > 30:
            FC = self.Framecube[0:16]
            self.flag = True
            return np.array(FC), str(self.x), len(self.Framecube)
        else:
            return "Preparing", str(self.x), len(self.Framecube)
        

In [22]:
# cap = cv2.VideoCapture(1)
import time
# ipcam = ipcamCapture(int(0))
ipcam = ipcamCapture(0)
predstart = time.time()

Gesture = [
           'Swiping Left',
           'Swiping Right', 
           'Swiping Down', 
           'Swiping Up', 
           'Turning Hand Clockwise', 
           'Turning Hand Counterclockwise'
           ]

# Stare query frame 
ipcam.start()

time.sleep(1)


while True:
    #  getframe 
    I = ipcam.getframe()
    # print(type(I))
    # print(I.shape)
    C, s, l = ipcam.getcubeframe()
    # print(type(C))
    if C == "Preparing":
        # print(C)
        # print(s)
        # print(l)
        pass
    else:
        # if l < 35:
        #     print('='*30)
        #     print(l)
        #     print('='*30)
        # else:
        #     print(l)

        # print(C.shape)
        # print(s)
        X = C[np.newaxis,:]
        # print(X.shape)
        pred = new_model.predict(X.astype('float32'))
        now = time.time()
        # print(np.max(pred[0]))
        if (pred[0] < 0.65).all():
            pass
        else:
            if np.max(pred[0]) > 0.88 and now-predstart > 1.5 :
                print('Gesture ==========================>',np.argmax(pred, axis=1))
                print('Gesture ==========================>',Gesture[int(np.argmax(pred, axis=1)[0])])
                print('Gesture ==========================>',np.max(pred[0]))
                KBC(np.argmax(pred, axis=1)[0])
                predstart = time.time()
            else:
                pass
    time.sleep(0.06)
    cv2.imshow('Image', I)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        cv2.destroyAllWindows()
        ipcam.stop()
        break

ipcam started!
Gesture ==========================> [1]
Gesture ==========================> Swiping Right
Gesture ==========================> 0.88871384
Gesture ==========================> [0]
Gesture ==========================> Swiping Left
Gesture ==========================> 0.95846707
Gesture ==========================> [3]
Gesture ==========================> Swiping Up
Gesture ==========================> 0.95698893
Gesture ==========================> [2]
Gesture ==========================> Swiping Down
Gesture ==========================> 0.98061496
Gesture ==========================> [5]
Gesture ==========================> Turning Hand Counterclockwise
Gesture ==========================> 0.95109063
Gesture ==========================> [2]
Gesture ==========================> Swiping Down
Gesture ==========================> 0.90783405
Gesture ==========================> [4]
Gesture ==========================> Turning Hand Clockwise
Gesture ==========================> 0.9321058
Gesture 

ipcam stopped!


In [10]:
# ipcam.stop()

ipcam stopped!
